In [ ]:
import certifi
import ssl
import geopy.geocoders
from geopy.geocoders import GoogleV3

ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx

In [ ]:
f = open("gmaps_key.txt", "r") 
key = f.read()

In [ ]:
geolocator = GoogleV3(api_key=key, user_agent="onhb-geo-scrapper", timeout=3)

In [ ]:
location = geolocator.geocode("Campinas, SP, Brazil")
print((location.latitude, location.longitude))

In [ ]:
import json

#with open('data.json') as json_file:  
#  data = json.load(json_file)

Merge two JSON files

In [ ]:
pd = '../p5js/data/processed_data.json'

with open(pd) as json_file:  
  data = json.load(json_file)
  
with open('new_data.json') as json_file:
  new_data = json.load(json_file)
  
print(f'old data started with {len(data["data"])}')

for updated_data in new_data['data']:
  city_exists = False
  for old_data in data['data']:
    if updated_data['cidade'] == old_data['cidade']:
      city_exists = True
      old_data['qtd'] = updated_data['qtd']
      break
  if not city_exists:
    data['data'].append(updated_data)
    print(f'could not find {updated_data["cidade"]}')
    
print(f'old data ended with {len(data["data"])}')

In [ ]:
# checks if n is equal in both json objects
c = 0
for nd in new_data['data']:
  c += int(nd['qtd'])
print(f'the new data count is {c}')

c = 0
for updated_data in data['data']:
  c += int(updated_data['qtd'])
print(f'the updated data count is {c}')

...and fill out missing longitude and latitudes.

In [ ]:
i = 1
suc = 0

for city in data['data']:
  if suc < 50:
    s = city['cidade'] + ', ' + city['uf'] + ', Brazil'
    if city['lat'] == '':
      try:
        location = geolocator.geocode(s)
      except Exception as e:
        print(f'#{i} -> {s} -> ERROR: {str(e)}')
      else:
        print(f'#{i} -> {s} -> {location.latitude} / {location.longitude}')
        city['lat'] = location.latitude
        city['long'] = location.longitude
        suc += 1
    else:
      print(f'#{i} -> already has {s}')
    i += 1
  else:
    suc = 0
    print('...will now save')
    with open(pd, 'w') as outfile:  
      json.dump(data, outfile, indent=4, sort_keys=True)
  
with open(pd, 'w') as outfile:  
    json.dump(data, outfile, indent=4, sort_keys=True)